# Different Ways of Using LLM APIs

### 01. OpenAI APIs
    - You can use OpenAI APIs directly by using OpenAI API key
    - OpenAI keys will available in your OpenAI account

### 02. Together AI APIs
    - Instead of using OpenAI API key, you can use Together AI API key
    - Together AI has hosted more than 100+ Models in both closed and open source

### 03. Azure APIs
    - You can access OpenAI APIs through Azurem instead of using OpenAI API directly
    - This is the best way to use OpenAI APIs in production
    - It will secure your data you input via prompting
    

# Prompt Engineering Basics

In [1]:
import os, yaml, together
from openai import OpenAI, ChatCompletion

In [2]:
with open('credential.yaml') as f:
    credentials = yaml.load(f, Loader=yaml.FullLoader)

os.environ['TOGETHER_AI_API'] = credentials['TOGETHER_AI_API']
os.environ['OPENAI_API_KEY'] = credentials['OPENAI_API_KEY']

### A) Basic Prompting

#### 01. Using OpenAI

In [3]:
client = OpenAI()

In [4]:
def completion_model(USER_MESSAGE):
    response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": USER_MESSAGE}
    ],
    temperature=0,
    max_tokens=25
    )
    return str(response.choices[0].message.content)

#### 02. Using Together AI

In [ ]:
together.api_key = os.environ["TOGETHER_AI_API"]

# list available models and descriptons
models = together.Models.list()
for m in models:
    print(m['name'])

In [ ]:
together.Models.start("togethercomputer/llama-2-13b-chat")

In [30]:
def completion_model_together(USER_MESSAGE):
    output = together.Complete.create(
                                    USER_MESSAGE,
                                    model="togethercomputer/llama-2-13b-chat",
                                    max_tokens=20,
                                    temperature=0,
                                    )
    text = output['output']['choices'][0]['text'].replace('\n', '   ').strip()
    return text

In [31]:
completion_model_together('Who is the president of the United States?')

'The current president of the United States is Joe Biden. He was inaugurated on January'

### B) Zero-Shot Prompting

In [12]:
def completion_model(USER_MESSAGE):
    response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": USER_MESSAGE}
    ],
    temperature=0,
    max_tokens=100
    )
    return str(response.choices[0].message.content)

# def completion_model(USER_MESSAGE):
#     response = client.chat.completions.create(
#     model="gpt-3.5-turbo",
#     messages=[
#         {"role": "system", "content": "You are a helpful sentiment analyzer. Classify the text into neutral, negative or positive."},
#         {"role": "user", "content": USER_MESSAGE}
#     ],
#     temperature=0,
#     max_tokens=10
#     )
#     return str(response.choices[0].message.content)

In [32]:
completion_model('''
Classify the text into neutral, negative or positive. 

Text: I think the vacation is okay.
Sentiment:
''')

'Neutral'

### C) Few-Shot Prompting

In [33]:
def completion_model(USER_MESSAGE):
    response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": USER_MESSAGE}
    ],
    temperature=0,
    max_tokens=10
    )
    return str(response.choices[0].message.content)

In [40]:
completion_model('''
                 
Classify the TEXT into neutral, negative or positive. use below examples for few shots. only return neutral, negative or positive
                 
This is awesome! // Positive
This is bad! // Negative
That movie was ok. // Neutral
                 

The day was not nice.
''')

'Negative'

### D) COT Prompting

In [41]:
def completion_model(USER_MESSAGE):
    response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": USER_MESSAGE}
    ],
    temperature=0,
    max_tokens=1000
    )
    return str(response.choices[0].message.content)

In [42]:
completion_model('''               
I went to the market and bought 10 apples. 
I gave 2 apples to the neighbor and 2 to the repairman.
I then went and bought 5 more apples and ate 1.             
How many apples did I remain with if I neglect the one I ate? Do not include the method of solving the problem.
                
Answer : 11 Apples 
                 
I went to the shop with 10 bags. each bag can store 3 fruits 
I bought fruits until all the bags were full. I gave 5 fruits to my friend and ate 1.
How many fruits did I remain with if I neglect the one I ate? Do not include the method of solving the problem. 

Answer :
''')

'29 fruits'

In [27]:
print(completion_model('''               
I went to the market and bought 10 apples. 
I gave 2 apples to the neighbor and 2 to the repairman.
I then went and bought 5 more apples and ate 1.             
How many apples did I remain with if I neglect the one I ate? Do not include the method of solving the problem.
                
Answer : I started with 10 apples. 
         After giving 2 to the neighbor and 2 to the repairman. 10 - 2 - 2 = 6.
         I bought 5 more apples. 6 + 5 = 11. I ate 1 but since we are neglecting it, I remained with 11 apples.
                 
I went to the shop with 10 bags. each bag can store 3 fruits 
I bought fruits until all the bags were full. I gave 5 fruits to my friend and ate 1.
How many fruits did I remain with if I neglect the one I ate? Do not include the method of solving the problem. 

Answer :
'''))

You started with 10 bags, and each bag can store 3 fruits. So you had a total of 10 x 3 = 30 fruits.
After giving 5 fruits to your friend, you had 30 - 5 = 25 fruits.
You then ate 1 fruit, but since we are neglecting it, you remained with 25 fruits.


### E) Self-Consistency

In [43]:
def completion_model(USER_MESSAGE):
    response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": USER_MESSAGE}
    ],
    temperature=0.9,
    max_tokens=100
    )
    return str(response.choices[0].message.content)


In [44]:
print(completion_model('''               
I went to the market and bought 10 apples. 
I gave 2 apples to the neighbor and 2 to the repairman.
I then went and bought 5 more apples and ate 1.             
How many apples did I remain with if I neglect the one I ate? Do not include the method of solving the problem.
                
Answer : I started with 10 apples. 
         After giving 2 to the neighbor and 2 to the repairman. 10 - 2 - 2 = 6.
         I bought 5 more apples. 6 + 5 = 11. I ate 1 but since we are neglecting it, I remained with 11 apples.
                 
I went to the shop with 10 bags. each bag can store 3 fruits 
I bought fruits until all the bags were full. I gave 5 fruits to my friend and ate 1.
How many fruits did I remain with if I neglect the one I ate? Do not include the method of solving the problem. 

Answer :
'''))

You started with 10 bags, each bag can store 3 fruits. So you had 10 x 3 = 30 fruits at the beginning.
You filled all the bags with fruits, so you used all 30 fruits.
Then you gave 5 fruits to your friend and ate 1, so you used 5 + 1 = 6 fruits.
Since we are neglecting the one you ate, you remained with 30 - 6 = 24 fruits.


In [45]:
print(completion_model('''               
I went to the market and bought 10 apples. 
I gave 2 apples to the neighbor and 2 to the repairman.
I then went and bought 5 more apples and ate 1.             
How many apples did I remain with if I neglect the one I ate? Do not include the method of solving the problem.
                
Answer : I started with 10 apples. 
         After giving 2 to the neighbor and 2 to the repairman. 10 - 2 - 2 = 6.
         I bought 5 more apples. 6 + 5 = 11. I ate 1 but since we are neglecting it, I remained with 11 apples.
                 
I went to the shop with 10 bags. each bag can store 3 fruits 
I bought fruits until all the bags were full. I gave 5 fruits to my friend and ate 1.
How many fruits did I remain with if I neglect the one I ate? Do not include the method of solving the problem. 

Answer :
'''))

You had 10 bags and each bag can store 3 fruits, so you had a total capacity of 10 bags * 3 fruits/bag = 30 fruits.
You filled all the bags with fruits, so you had 30 fruits.
You gave 5 fruits to your friend, so you had 30 fruits - 5 fruits = 25 fruits left.
You ate 1 fruit, but since we are neglecting it, you remained with 25 fruits.


### E) ToT Prompting

Imagine three different experts are answering this question. </br>
All experts will write down 1 step of their thinking, </br>
then share it with the group. </br>
Then all experts will go on to the next step, etc. </br>
If any expert realises they're wrong at any point then they leave. </br>
The question is...